# Importação e Definição de métodos

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from segmentation_models_pytorch import Unet
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import torchmetrics
from medpy.metric.binary import hd, asd
import time


import ssl
import urllib.request

from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

c:\Users\Vitor\Desktop\TI0160-VisaoComputacionalAplicadaASaude-UFC-2024.1\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DatasetBreastCancer(Dataset):
    def __init__(self, image_paths, mask_paths, image_size=(256, 256)):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.image_size = image_size

        # Define transformations for grayscale images and masks
        self.image_transform = transforms.Compose([
            transforms.Resize(self.image_size),      # Resize image to fixed size
            transforms.ToTensor(),                   # Convert image to PyTorch tensor
        ])
        
        self.mask_transform = transforms.Compose([
            transforms.Resize(self.image_size, interpolation=Image.NEAREST),  # Resize mask using nearest neighbor
            transforms.ToTensor(),                                          # Convert mask to tensor
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Open image and mask in grayscale
        image = Image.open(self.image_paths[idx]).convert('L')  # 'L' ensures grayscale (single channel)
        mask = Image.open(self.mask_paths[idx]).convert('L')    # Mask is also in grayscale

        # Apply transformations
        image = self.image_transform(image)
        mask = self.mask_transform(mask)

        return image, mask

In [3]:
import os
from sklearn.model_selection import train_test_split

base_dir = '../DATA/raw'
categories = ['benign', 'malignant', 'normal']

train_image_paths = []
val_image_paths = []
train_mask_paths = []
val_mask_paths = []

for category in categories:
    category_dir = os.path.join(base_dir, category)
    category_image_paths = []
    category_mask_paths = []
    
    for fname in os.listdir(category_dir):
        if fname.endswith('_mask.png'):
            category_mask_paths.append(os.path.join(category_dir, fname))
        else:
            category_image_paths.append(os.path.join(category_dir, fname))
    
    # Dividir os caminhos em conjuntos de treino e validação para cada categoria
    train_img, val_img, train_mask, val_mask = train_test_split(
        category_image_paths, category_mask_paths, test_size=0.2, random_state=42
    )
    
    train_image_paths.extend(train_img)
    val_image_paths.extend(val_img)
    train_mask_paths.extend(train_mask)
    val_mask_paths.extend(val_mask)

# Criar instâncias dos datasets de treino e validação
train_dataset = DatasetBreastCancer(train_image_paths, train_mask_paths)
val_dataset = DatasetBreastCancer(val_image_paths, val_mask_paths)

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context

# Definindo o dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, masks in dataloader:
            inputs = inputs.to(device)
            masks = masks.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}")

In [6]:
from scipy.spatial.distance import directed_hausdorff
import numpy as np
import matplotlib.pyplot as plt

def dice_coefficient(preds, targets, threshold=0.5, epsilon=1e-6):
    # Apply threshold to get binary predictions
    preds = (preds > threshold).float()
    
    # Flatten the tensors to compare corresponding pixels
    preds = preds.view(-1)
    targets = targets.view(-1)
    
    # Compute intersection and union
    intersection = (preds * targets).sum()
    union = preds.sum() + targets.sum()
    
    # Compute Dice score
    dice = (2. * intersection + epsilon) / (union + epsilon)
    
    return dice

def jaccard_index(preds, targets, threshold=0.5, epsilon=1e-6):
    # Apply threshold to get binary predictions
    preds = (preds > threshold).float()
    
    # Flatten the tensors to compare corresponding pixels
    preds = preds.view(-1)
    targets = targets.view(-1)
    
    # Compute intersection and union
    intersection = (preds * targets).sum()
    union = (preds + targets).sum() - intersection
    
    # Compute Jaccard Index
    jaccard = (intersection + epsilon) / (union + epsilon)
    
    return jaccard

def balanced_average_hausdorff_distance(predictions: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    """
    Calculates the balanced average Hausdorff distance between predictions and targets.

    Parameters:
        predictions (torch.Tensor):
            The predicted tensor.
        targets (torch.Tensor):
            The target tensor.

    Returns:
        torch.Tensor:
            The balanced average Hausdorff distance.
    """

    batch_size, channels, *_ = targets.shape
    values = torch.zeros((batch_size, channels))

    for batch in range(batch_size):
        for channel in range(channels):
            targets_, predictions_ = (
                targets[batch][channel].detach().cpu(),
                predictions[batch][channel].detach().cpu(),
            )
            values[batch, channel] = (
                directed_hausdorff(targets_, predictions_)[0]
                + directed_hausdorff(predictions_, targets_)[0]
                * predictions_.view(1, -1).size(1)
                / targets_.view(1, -1).size(1)
            ) / 2

    return values.mean()

def handle_empty_array(array: np.ndarray) -> np.ndarray:
    """
    Handle empty arrays by filling them with a default value if they contain only zeros.

    Args:
        array (np.ndarray):
            Input array.

    Returns:
        np.ndarray:
            Array with zeros replaced by a default value if the array was empty.
    """

    if np.count_nonzero(array) == 0:
        array[0, 0] = 1

    return array

def average_surface_distance(
    predictions: torch.Tensor, targets: torch.Tensor, threshold: float = 0.5, connectivity: int = 1
) -> torch.Tensor:
    """
    Calculates the average surface distance between two binary segmentation masks.

    Parameters:
        predictions (torch.Tensor):
            The first binary segmentation mask tensor.
        targets (torch.Tensor):
            The second binary segmentation mask tensor.
        threshold (float, optional):
            Threshold value to convert predictions to binary. Defaults to 0.5.

    Returns:
        torch.Tensor:
            The average surface distance.
    """

    predictions_, targets_ = (predictions > threshold).detach().cpu().numpy(), targets.detach().cpu().numpy()

    predictions_, targets_ = handle_empty_array(predictions_), handle_empty_array(targets_)

    return asd(predictions_, targets_)

def evaluate_model(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    dice_scores = []
    jaccard_scores = []
    hausdorff_distances = []
    surface_distances = []

    with torch.no_grad():  # Disable gradient calculations
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass to get model predictions
            outputs = model(inputs)
            
            # Apply sigmoid or softmax if needed (for binary/multi-class segmentation)
            outputs = torch.sigmoid(outputs) if outputs.shape[1] == 1 else torch.softmax(outputs, dim=1)
            
            # Calculate dice score per batch
            # Calculate dice score and jaccard index per batch
            dice = dice_coefficient(outputs, targets)
            jaccard = jaccard_index(outputs, targets)
            hausdorff = balanced_average_hausdorff_distance(outputs, targets)
            surface = average_surface_distance(outputs, targets)
            
            dice_scores.append(dice.item())
            jaccard_scores.append(jaccard.item())
            hausdorff_distances.append(hausdorff)
            surface_distances.append(surface)
    
    # Calculate the mean Dice and Jaccard coefficients
    mean_dice = sum(dice_scores) / len(dice_scores)
    mean_jaccard = sum(jaccard_scores) / len(jaccard_scores)
    mean_hausdorff = sum(hausdorff_distances) / len(hausdorff_distances)
    mean_surface = sum(surface_distances) / len(surface_distances)

    print(f"Mean Dice Coefficient: {mean_dice:.4f}")
    print(f"Mean Jaccard Index: {mean_jaccard:.4f}")
    print(f"Mean Balanced Average Hausdorff Distance: {mean_hausdorff:.4f}")
    print(f"Mean Average Surface Distance: {mean_surface:.4f}")

# Encoder: DenseNet201 - Decoder: UNET

In [7]:
import os

# Função de pré-processamento
preprocess_input = get_preprocessing_fn('densenet201', pretrained='imagenet')

# Definindo o modelo
densenet201_unet_model = Unet(encoder_name="densenet201", encoder_weights="imagenet", in_channels=1, classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(densenet201_unet_model.parameters(), lr=1e-4)

model_path = 'densenet201_unet_model.pth'

if os.path.exists(model_path):
    densenet201_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado a partir do disco.")
else:
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    train_model(densenet201_unet_model, train_dataloader, criterion, optimizer, num_epochs=25)
    evaluate_model(densenet201_unet_model, val_dataloader, device)
    
    # Salvando o modelo
    torch.save(densenet201_unet_model.state_dict(), model_path)
    print("Modelo treinado e salvo no disco.")

Epoch 0/24, Loss: 0.6877
Epoch 1/24, Loss: 0.4273
Epoch 2/24, Loss: 0.3333
Epoch 3/24, Loss: 0.2834
Epoch 4/24, Loss: 0.2490
Epoch 5/24, Loss: 0.2236
Epoch 6/24, Loss: 0.2030
Epoch 7/24, Loss: 0.1873
Epoch 8/24, Loss: 0.1679
Epoch 9/24, Loss: 0.1466
Epoch 10/24, Loss: 0.1295
Epoch 11/24, Loss: 0.1197
Epoch 12/24, Loss: 0.1066
Epoch 13/24, Loss: 0.0988
Epoch 14/24, Loss: 0.0892
Epoch 15/24, Loss: 0.0812
Epoch 16/24, Loss: 0.0733
Epoch 17/24, Loss: 0.0679
Epoch 18/24, Loss: 0.0631
Epoch 19/24, Loss: 0.0593
Epoch 20/24, Loss: 0.0542
Epoch 21/24, Loss: 0.0510
Epoch 22/24, Loss: 0.0485
Epoch 23/24, Loss: 0.0454
Epoch 24/24, Loss: 0.0427
Mean Dice Coefficient: 0.7967
Mean Jaccard Index: 0.6739
Mean Balanced Average Hausdorff Distance: 3.4062
Mean Average Surface Distance: 0.4905
Modelo treinado e salvo no disco.


# Encoder: Resnet152 - Decoder: UNET


In [8]:
# Função de pré-processamento
preprocess_input = get_preprocessing_fn('resnet152', pretrained='imagenet')

# Definindo o modelo
resnet152_unet_model = Unet(encoder_name="resnet152", encoder_weights="imagenet", in_channels=1, classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(resnet152_unet_model.parameters(), lr=1e-4)

model_path = 'resnet152_unet_model.pth'

if os.path.exists(model_path):
    resnet152_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado a partir do disco.")
else:
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    train_model(resnet152_unet_model, train_dataloader, criterion, optimizer, num_epochs=25)
    evaluate_model(resnet152_unet_model, val_dataloader, device)
    
    # Salvando o modelo
    torch.save(resnet152_unet_model.state_dict(), model_path)
    print("Modelo treinado e salvo no disco.")

Epoch 0/24, Loss: 0.5489
Epoch 1/24, Loss: 0.3534
Epoch 2/24, Loss: 0.2680
Epoch 3/24, Loss: 0.2225
Epoch 4/24, Loss: 0.1904
Epoch 5/24, Loss: 0.1656
Epoch 6/24, Loss: 0.1489
Epoch 7/24, Loss: 0.1354
Epoch 8/24, Loss: 0.1203
Epoch 9/24, Loss: 0.1071
Epoch 10/24, Loss: 0.0966
Epoch 11/24, Loss: 0.0870
Epoch 12/24, Loss: 0.0784
Epoch 13/24, Loss: 0.0704
Epoch 14/24, Loss: 0.0646
Epoch 15/24, Loss: 0.0589
Epoch 16/24, Loss: 0.0544
Epoch 17/24, Loss: 0.0508
Epoch 18/24, Loss: 0.0475
Epoch 19/24, Loss: 0.0441
Epoch 20/24, Loss: 0.0411
Epoch 21/24, Loss: 0.0376
Epoch 22/24, Loss: 0.0358
Epoch 23/24, Loss: 0.0333
Epoch 24/24, Loss: 0.0329
Mean Dice Coefficient: 0.7017
Mean Jaccard Index: 0.5767
Mean Balanced Average Hausdorff Distance: 3.4220
Mean Average Surface Distance: 1.7786
Modelo treinado e salvo no disco.


# Encoder: VGG19 - Decoder: UNET

In [9]:
# Função de pré-processamento
preprocess_input = get_preprocessing_fn('vgg19', pretrained='imagenet')

# Definindo o modelo
vgg19_unet_model = Unet(encoder_name="vgg19", encoder_weights="imagenet", in_channels=1, classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(vgg19_unet_model.parameters(), lr=1e-4)

model_path = 'vgg19_unet_model.pth'

if os.path.exists(model_path):
    vgg19_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado a partir do disco.")
else:
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    train_model(vgg19_unet_model, train_dataloader, criterion, optimizer, num_epochs=25)
    evaluate_model(vgg19_unet_model, val_dataloader, device)
    
    # Salvando o modelo
    torch.save(vgg19_unet_model.state_dict(), model_path)
    print("Modelo treinado e salvo no disco.")

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\Vitor/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:50<00:00, 11.5MB/s] 


Epoch 0/24, Loss: 0.6300
Epoch 1/24, Loss: 0.4263
Epoch 2/24, Loss: 0.3444
Epoch 3/24, Loss: 0.2990
Epoch 4/24, Loss: 0.2667
Epoch 5/24, Loss: 0.2337
Epoch 6/24, Loss: 0.2125
Epoch 7/24, Loss: 0.1957
Epoch 8/24, Loss: 0.1696
Epoch 9/24, Loss: 0.1472
Epoch 10/24, Loss: 0.1337
Epoch 11/24, Loss: 0.1243
Epoch 12/24, Loss: 0.1137
Epoch 13/24, Loss: 0.1043
Epoch 14/24, Loss: 0.0944
Epoch 15/24, Loss: 0.0901
Epoch 16/24, Loss: 0.0887
Epoch 17/24, Loss: 0.0786
Epoch 18/24, Loss: 0.0674
Epoch 19/24, Loss: 0.0623
Epoch 20/24, Loss: 0.0585
Epoch 21/24, Loss: 0.0590
Epoch 22/24, Loss: 0.0570
Epoch 23/24, Loss: 0.0495
Epoch 24/24, Loss: 0.0461
Mean Dice Coefficient: 0.8016
Mean Jaccard Index: 0.6793
Mean Balanced Average Hausdorff Distance: 3.2914
Mean Average Surface Distance: 0.6692
Modelo treinado e salvo no disco.


# Encoder: Xception - Decoder: UNET

In [10]:
# Função de pré-processamento
preprocess_input = get_preprocessing_fn('xception', pretrained='imagenet')

# Definindo o modelo
xception_unet_model = Unet(encoder_name="xception", encoder_weights="imagenet", in_channels=1, classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(xception_unet_model.parameters(), lr=1e-4)

model_path = 'xception_unet_model.pth'

if os.path.exists(model_path):
    xception_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado a partir do disco.")
else:
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    train_model(xception_unet_model, train_dataloader, criterion, optimizer, num_epochs=25)
    evaluate_model(xception_unet_model, val_dataloader, device)
    
    # Salvando o modelo
    torch.save(xception_unet_model.state_dict(), model_path)
    print("Modelo treinado e salvo no disco.")

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth" to C:\Users\Vitor/.cache\torch\hub\checkpoints\xception-43020ad28.pth
100%|██████████| 87.4M/87.4M [04:17<00:00, 355kB/s]


Epoch 0/24, Loss: 0.4246
Epoch 1/24, Loss: 0.2820
Epoch 2/24, Loss: 0.2160
Epoch 3/24, Loss: 0.1736
Epoch 4/24, Loss: 0.1453
Epoch 5/24, Loss: 0.1196
Epoch 6/24, Loss: 0.1046
Epoch 7/24, Loss: 0.0924
Epoch 8/24, Loss: 0.0815
Epoch 9/24, Loss: 0.0730
Epoch 10/24, Loss: 0.0633
Epoch 11/24, Loss: 0.0582
Epoch 12/24, Loss: 0.0541
Epoch 13/24, Loss: 0.0496
Epoch 14/24, Loss: 0.0446
Epoch 15/24, Loss: 0.0427
Epoch 16/24, Loss: 0.0406
Epoch 17/24, Loss: 0.0376
Epoch 18/24, Loss: 0.0341
Epoch 19/24, Loss: 0.0317
Epoch 20/24, Loss: 0.0299
Epoch 21/24, Loss: 0.0278
Epoch 22/24, Loss: 0.0263
Epoch 23/24, Loss: 0.0253
Epoch 24/24, Loss: 0.0242
Mean Dice Coefficient: 0.7038
Mean Jaccard Index: 0.5809
Mean Balanced Average Hausdorff Distance: 3.3270
Mean Average Surface Distance: 1.1955
Modelo treinado e salvo no disco.


# Encoder: InceptionResNetV2 - Decoder: UNET

In [11]:
# Função de pré-processamento
preprocess_input = get_preprocessing_fn('inceptionresnetv2', pretrained='imagenet')

# Definindo o modelo
inceptionresnetv2_unet_model = Unet(encoder_name="inceptionresnetv2", encoder_weights="imagenet", in_channels=1, classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(inceptionresnetv2_unet_model.parameters(), lr=1e-4)

model_path = 'inceptionresnetv2_unet_model.pth'

if os.path.exists(model_path):
    inceptionresnetv2_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado a partir do disco.")
else:
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    train_model(inceptionresnetv2_unet_model, train_dataloader, criterion, optimizer, num_epochs=25)
    evaluate_model(inceptionresnetv2_unet_model, val_dataloader, device)
    
    # Salvando o modelo
    torch.save(inceptionresnetv2_unet_model.state_dict(), model_path)
    print("Modelo treinado e salvo no disco.")

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth" to C:\Users\Vitor/.cache\torch\hub\checkpoints\inceptionresnetv2-520b38e4.pth
100%|██████████| 213M/213M [10:29<00:00, 356kB/s] 


Epoch 0/24, Loss: 0.6031
Epoch 1/24, Loss: 0.4400
Epoch 2/24, Loss: 0.3451
Epoch 3/24, Loss: 0.2777
Epoch 4/24, Loss: 0.2344
Epoch 5/24, Loss: 0.2037
Epoch 6/24, Loss: 0.1760
Epoch 7/24, Loss: 0.1534
Epoch 8/24, Loss: 0.1347
Epoch 9/24, Loss: 0.1190
Epoch 10/24, Loss: 0.1056
Epoch 11/24, Loss: 0.0950
Epoch 12/24, Loss: 0.0868
Epoch 13/24, Loss: 0.0780
Epoch 14/24, Loss: 0.0703
Epoch 15/24, Loss: 0.0633
Epoch 16/24, Loss: 0.0571
Epoch 17/24, Loss: 0.0523
Epoch 18/24, Loss: 0.0484
Epoch 19/24, Loss: 0.0452
Epoch 20/24, Loss: 0.0428
Epoch 21/24, Loss: 0.0407
Epoch 22/24, Loss: 0.0372
Epoch 23/24, Loss: 0.0343
Epoch 24/24, Loss: 0.0329
Mean Dice Coefficient: 0.7112
Mean Jaccard Index: 0.5908
Mean Balanced Average Hausdorff Distance: 3.2447
Mean Average Surface Distance: 1.2906
Modelo treinado e salvo no disco.


# Encoder: MobileNetV2 - Decoder: UNET

In [12]:
# Função de pré-processamento
preprocess_input = get_preprocessing_fn('mobilenet_v2', pretrained='imagenet')

# Definindo o modelo
mobilenetv2_unet_model = Unet(encoder_name="mobilenet_v2", encoder_weights="imagenet", in_channels=1, classes=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(mobilenetv2_unet_model.parameters(), lr=1e-4)

model_path = 'mobilenetv2_unet_model.pth'

if os.path.exists(model_path):
    mobilenetv2_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado a partir do disco.")
else:
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    train_model(mobilenetv2_unet_model, train_dataloader, criterion, optimizer, num_epochs=25)
    evaluate_model(mobilenetv2_unet_model, val_dataloader, device)
    
    # Salvando o modelo
    torch.save(mobilenetv2_unet_model.state_dict(), model_path)
    print("Modelo treinado e salvo no disco.")

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\Vitor/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:01<00:00, 11.8MB/s]


Epoch 0/24, Loss: 0.6342
Epoch 1/24, Loss: 0.4535
Epoch 2/24, Loss: 0.3679
Epoch 3/24, Loss: 0.3081
Epoch 4/24, Loss: 0.2635
Epoch 5/24, Loss: 0.2270
Epoch 6/24, Loss: 0.2051
Epoch 7/24, Loss: 0.1785
Epoch 8/24, Loss: 0.1591
Epoch 9/24, Loss: 0.1389
Epoch 10/24, Loss: 0.1272
Epoch 11/24, Loss: 0.1138
Epoch 12/24, Loss: 0.1028
Epoch 13/24, Loss: 0.0922
Epoch 14/24, Loss: 0.0841
Epoch 15/24, Loss: 0.0781
Epoch 16/24, Loss: 0.0724
Epoch 17/24, Loss: 0.0671
Epoch 18/24, Loss: 0.0624
Epoch 19/24, Loss: 0.0597
Epoch 20/24, Loss: 0.0567
Epoch 21/24, Loss: 0.0516
Epoch 22/24, Loss: 0.0473
Epoch 23/24, Loss: 0.0445
Epoch 24/24, Loss: 0.0423
Mean Dice Coefficient: 0.6831
Mean Jaccard Index: 0.5531
Mean Balanced Average Hausdorff Distance: 3.5195
Mean Average Surface Distance: 1.7420
Modelo treinado e salvo no disco.
